# OCR experiment

This notebook compares the OCR results embedded in the textbook scans to those of eric's api, and MS oxford alone

bash curl -H "Content-Type: application/json" -X POST -d '{"url":" http://dgmde20.com/img/lorem-ipsum-2.png"}' http://dev-ocr.ai2:/v1/ocr

In [84]:
%%capture
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, OrderedDict
%load_ext autoreload
%autoreload 2

In [3]:
import cv2
import pprint
import pickle
import json
import requests
import io

In [28]:
import pdfextraction.ocr_pipeline

In [43]:
# def query_vision_ocr(image_url):
#     api_entry_point = 'http://10.12.2.9:8000/v1/ocr'
#     header = {'Content-Type':'application/json'}
    
#     with open(page_image_path, 'rb') as f:
#         encoded_image_str = base64.b64encode(f.read())
        
#     request_data = {
#         'image': encoded_image_str
#         }
#     json_data = json.dumps(request_data)
# #     response = requests.post(api_entry_point, data=json_data, headers=header)
#     return json_data

In [20]:
li_image_url = 'http://dgmde20.com/img/lorem-ipsum-2.png'
image_url = 'https://s3-us-west-2.amazonaws.com/ai2-vision-turk-data/textbook-pages-test/test_page.jpeg'

In [8]:
ocr_res = pdfextraction.ocr_pipeline.query_vision_ocr(image_url)

In [134]:
# with open ('ocr_res.json', 'w') as f:
#     json.dump(ocr_res, f)

In [6]:
with open ('ocr_res.json', 'r') as f:
    ocr_res = json.load( f)

In [7]:
ocr_res

In [153]:
for det in ocr_res['detections']:
    print(det['value'], '\n')

In [19]:
point_to_tuple = lambda box: tuple(OrderedDict(sorted(box.items())).values())
get_bbox_tuples = lambda detection: map(point_to_tuple, detection['rectangle'])

In [15]:
def display_page(raw_page_img, page_layout):
    page_png_stream, y_height = make_png_stream(raw_page_img)
    page_img = make_open_cv_img(page_png_stream)
    for box in page_layout._objs:
        lr, ul = get_bbox_tuple(box, y_height)
        try:
            # print box.get_text()
            pass
        except AttributeError:
            pass
        cv2.rectangle(page_img, ul, lr, color=random_color(), thickness=2)
    display(Image.fromarray(page_img, 'RGB'))


def draw_pdf_with_boxes(book_file, page_range, word_margin=0.1, line_overlap=0.5, char_margin=2.0,
                        line_margin=0.5, boxes_flow=0.5):
    if page_range:
        page_range = map(lambda x: x - 1, page_range)
        suffix = '[{}-{}]'.format(page_range[0], page_range[1])
        raw_multi_pdf = WImage(filename=book_file + suffix)
    else:
        raw_multi_pdf = WImage(filename=book_file)
        
    doc_page_layouts = make_page_layouts(book_file, page_range,
                                         line_overlap,
                                         char_margin,
                                         line_margin,
                                         word_margin,
                                         boxes_flow)
    page_images = raw_multi_pdf.sequence
    for page_n in range(len(page_images)):
        display_page(page_images[page_n], doc_page_layouts[page_n])

In [16]:
import base64
from wand.image import Image as WImage
from IPython.display import display
import PIL.Image as Image

In [17]:
page_image_path = './page_sample.png'
with open(page_image_path, 'rb') as f:
        encoded_image_str = f.read()

In [25]:
req = requests.get(image_url)
tpi = Image.open(io.BytesIO(req.content))
# tpi.thumbnail((1000,1000))

In [8]:
def random_color():
    import random
    return random.randint(0,255), random.randint(0,255), random.randint(0,255)

In [9]:
def display_page(raw_page_img):
    page_png_stream, y_height = make_png_stream(raw_page_img)
    page_img = make_open_cv_img(page_png_stream)
    for box in page_layout._objs:
        lr, ul = get_bbox_tuple(box, y_height)
        try:
            # print box.get_text()
            pass
        except AttributeError:
            pass
        cv2.rectangle(page_img, ul, lr, color=random_color(), thickness=2)
    display(Image.fromarray(page_img, 'RGB'))

In [26]:
pil_image = tpi.convert('RGB') 
open_cv_image = np.array(pil_image) 
open_cv_image = open_cv_image[:, :, ::-1].copy() 

In [27]:
for box in ocr_res['detections']:
    lr, ul = get_bbox_tuples(box)
    cv2.rectangle(open_cv_image, ul, lr, color=random_color(), thickness=2)
display(Image.fromarray(open_cv_image, 'RGB'))

    rties": {
                        "box_id": {
                            "type": "String"
                        },
                        "category": {
                                "enum": ["header/topic", "definition", "discussion", "question", "question", "diagram", "unlabeled"]
                            },
                        "contents": {
                            "type": "string"
                        },
                        "score": {
                          "type": "number"
                        },
                        "rectangle": {
                            "type": "array",
                            "minItems": 2,
                            "maxItems": 2,
                            "items": {
                                "type": "array",
                                "minItems": 2,
                                "maxItems": 2,
                                "items": {
                                    "type": "integer",
                                },
                            },
                        },
                        "source": {
                            "type": "array",
                            "minItems": 2,
                            "maxItems": 2,
                            "items": {
                                "$schema": "http://json-schema.org/draft-04/schema#",
                                "title": "C Object",

                                "type": "object",
                                "required": ["id", "name"],

                                "properties": {
                                    "book_source": {
                                        "type": "string"
                                    },
                                    "page_n": {
                                        "type": "int"
                                    }
                                },
                                "additionalProperties": False
                            }
                        }
                    }
                }
            }
        },

In [10]:
import ordereddict

In [147]:
import jsonschema
from pdfextraction.annotation import page_schema

In [19]:
point_to_tuple = lambda box: tuple(OrderedDict(sorted(box.items())).values())
get_bbox_tuples = lambda detection: map(point_to_tuple, detection['rectangle'])

In [153]:
ids = 1
annotation = defaultdict(defaultdict)
for box in ocr_res['detections']:
    box_id = 'T' + str(ids)
    bounding_rectangle = get_bbox_tuples(box)
    annotation['text'][box_id] = {
        "box_id": 'M4',
        "category": "unlabeled", 
        "contents": box['value'],
        "score": box['score'],
        "rectangle": bounding_rectangle,
        "source": {
            "type": "object",
            "$schema": "http://json-schema.org/draft-04/schema",
            "additionalProperties": False,
            "properties": [
                {"book_source": "sb"}, 
                {"page_n": 149}
            ]
        }
    }
    ids += 1
    annotation_json = json.dumps(annotation)

In [154]:
aj1 = json.loads(annotation_json)

validator = jsonschema.Draft4Validator(page_schema)
validator.validate(aj1)

In [155]:
aj1